In [362]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import math
import csv
from tqdm import tqdm
import requests
import _thread
import threading


In [4]:
df = pd.read_pickle('./complaints.pkl')
df.complaints[0]['url']

'https://data.cityofchicago.org/resource/fypr-ksnz.csv?street_number=1%20and%20direction=%27S%27%20and%20street_name=%27FINANCIAL%27%20and%20street_type=%27PL%27'

response = None
for i in tqdm(range(df.shape[0])):
    if type(response) != type(pd.DataFrame):
            response = pd.read_csv(df.complaints[i]['url'])
    else:
        try:
            response.append(pd.read_csv(df.complaints[i]['url']))
        except:
            #passing timeouts
            print(f'error on complain {i}')

In [349]:
complaints =  pd.read_csv(df.complaints[0]['url'])[0:0]
complaints['complaints'] = pd.Series()
lock = threading.Lock()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [350]:
class myThread (threading.Thread):
    def __init__(self, threadID, name, counter, complaint_df):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
        self.complaint_df = complaint_df
    def run(self):
        print("Starting " + self.name)
        self.complaint_df = worker(self.threadID, self.complaint_df)
        global complaints
        lock.acquire()
        print(f'Size of df: {self.complaint_df.shape[0]}')
        complaints = complaints.append(self.complaint_df)
        lock.release()
        print ("Exiting " + self.name)



In [351]:
def worker(start,complaint_df):
    for row in range(start,100,8):
        if row % 100 == 0 :
            print(f'Completed {row} downloads',end='\r')
        if complaint_df is None:   
            complaint_df = pd.read_csv(df.complaints[row]['url'])
            complaint_df['complaints'] = pd.Series(dict)
            for j in range(complaint_df.shape[0]):
                complaint_df.complaints[j] = df.complaints[row]
        else:
            try:
                temp = pd.read_csv(df.complaints[row]['url'])
                temp['complaints'] = pd.Series(dict)
                for j in range(temp.shape[0]):
                    temp.complaints[j] = df.complaints[row]
                
                complaint_df = complaint_df.append(temp)
            except:
                 print(f'error on complaint: {row}')
    return complaint_df

In [352]:
c1,c2,c3,c4,c5,c6,c7,c8 = [None for i in range(8)]
thread1 = myThread(1, "Thread-1", 1, c1)
thread2 = myThread(2, "Thread-2", 2, c2)
thread3 = myThread(3, "Thread-3", 3, c3)
thread4 = myThread(4, "Thread-4", 4, c4)
thread5 = myThread(5, "Thread-5", 5, c5)
thread6 = myThread(6, "Thread-6", 6, c6)
thread7 = myThread(7, "Thread-7", 7, c7)
thread8 = myThread(8, "Thread-8", 8, c8)

In [353]:

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()
thread6.start()
thread7.start()
thread8.start()


Starting Thread-1
Starting Thread-2
Starting Thread-3
Starting Thread-4
Starting Thread-5
Starting Thread-6
Starting Thread-7
Starting Thread-8


In [357]:
complaints

,complaint_id,complaint_type,address,street_number,street_number_to,direction,street_name,street_type,inspector,complaint_date,complaint_detail,inspection_log,data_source,modified_date,latitude,longitude,location,complaints
0,DOECOMP5150,Air Pollution Work Order,1511 W NELSON ST,1511,NaN,W,NELSON,ST,34,1994-10-03T00:00:00.000,APARTMENT BUILDING FUMES COMI...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.936873,-87.666641,POINT (-87.66664076806524 41.93687258531489),{'url': 'https://data.cityofchicago.org/resour...
0,DOECOMP10602,Air Pollution Work Order,2323 S WESTERN AVE,2323,NaN,S,WESTERN,AVE,55,1999-10-18T00:00:00.000,RESIDENTIAL AREA VERY BAD F...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.849478,-87.685418,POINT (-87.68541790974605 41.84947815709742),{'url': 'https://data.cityofchicago.org/resour...
0,DOECOMP16971,Toxics Hazardous Materials Work Order,3350 W 48TH PL,3350,NaN,W,48TH,PL,14,2005-03-02T00:00:00.000,DIESLE FUEL SPILL. INSPECTOR OBSERVED THAT APP...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.805752,-87.708399,POINT (-87.70839894629215 41.80575180771991),{'url': 'https://data.cityofchicago.org/resour...
1,7320,ILLEGAL DUMPING WORK ORDER,3350 W 48TH PL,3350,NaN,W,48TH,PL,8386,2013-08-16T00:00:00.000,FORD F150 EXTENDED CAB WHITE DUMPED DEBRIS MAT...,[INSPECTION LOG #: 6841 16-AUG-13 11:15:00] I ...,DEPT. OF PUBLIC HEALTH,2013-08-16T00:00:00.000,41.805752,-87.708399,POINT (-87.70839894629215 41.80575180771991),{'url': 'https://data.cityofchicago.org/resour...
0,DOECOMP21746,Illegal Dumping Work Order,4441 N MILWAUKEE AVE,4441,NaN,N,MILWAUKEE,AVE,14,1994-12-07T00:00:00.000,PILES OF TRASH ALONG SIDE OF BUILDING VERBAL W...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.961701,-87.755229,POINT (-87.7552286416355 41.961700700121405),{'url': 'https://data.cityofchicago.org/resour...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DOECOMP12114,Service Stations/Storage Tanks Work Order,2501 N CALIFORNIA AVE,2501,NaN,N,CALIFORNIA,AVE,17,1994-04-27T00:00:00.000,CITATION ISSUED CA920669 DUE TO REFUESED TO RE...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.926683,-87.697318,POINT (-87.69731840917918 41.926683260829506),{'url': 'https://data.cityofchicago.org/resour...
0,DOECOMP19475,Noise Complaint,3924 N CICERO AVE,3924,NaN,N,CICERO,AVE,11,1995-08-16T00:00:00.000,MR. SUBMARINE,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.952310,-87.747434,POINT (-87.74743369959046 41.952310186986786),{'url': 'https://data.cityofchicago.org/resour...
1,DOECOMP19474,Noise Complaint,3924 N CICERO AVE,3924,NaN,N,CICERO,AVE,11,1995-05-23T00:00:00.000,MR. SUBMARINE NOISE FROM...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.952310,-87.747434,POINT (-87.74743369959046 41.952310186986786),{'url': 'https://data.cityofchicago.org/resour...
2,DOECOMP19473,Noise Complaint,3924 N CICERO AVE,3924,NaN,N,CICERO,AVE,11,1994-03-01T00:00:00.000,MR SUBMARINE RESAURANT NOISE USIN...,MORE INFORMATION MAY BE AVAILABLE IN THE CDPH ...,HISTORIC DEPT. OF ENVIRONMENT,2012-01-01T00:00:00.000,41.952310,-87.747434,POINT (-87.74743369959046 41.952310186986786),{'url': 'https://data.cityofchicago.org/resour...


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Size of df: 14
Exiting Thread-4


In [219]:
df.shape[0]

35116

In [251]:
df

,address,street_number,direction,street_name,street_type,complaints,inspections,latitude,longitude,location,permits,enforcement,tanks,neshaps_demolition_notices,holds
0,1 S FINANCIAL PL,1,S,FINANCIAL,PL,{'url': 'https://data.cityofchicago.org/resour...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000 S COMMERCIAL AVE,10000,S,COMMERCIAL,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.71330927944233,-87.55109645009955,"{'type': 'Point', 'coordinates': [-87.55109645...",NaN,NaN,NaN,NaN,NaN
2,10157 S WESTERN AVE,10157,S,WESTERN,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.70840185452616,-87.68153590856559,"{'type': 'Point', 'coordinates': [-87.68153590...",{'url': 'https://data.cityofchicago.org/resour...,NaN,NaN,NaN,NaN
3,1059 N WOLCOTT AVE,1059,N,WOLCOTT,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.90132042594509,-87.67461659752595,"{'type': 'Point', 'coordinates': [-87.67461659...",NaN,NaN,NaN,NaN,NaN
4,111 W 87TH ST,111,W,87TH,ST,{'url': 'https://data.cityofchicago.org/resour...,NaN,41.73615956889584,-87.62747171270124,"{'type': 'Point', 'coordinates': [-87.62747171...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35111,7742 S HOMAN AVE,7742,S,HOMAN,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.75208778573627,-87.70749209969989,"{'type': 'Point', 'coordinates': [-87.70749209...",NaN,NaN,NaN,NaN,NaN
35112,7414 S MAY ST,7414,S,MAY,ST,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.759140319710205,-87.65289476576204,"{'type': 'Point', 'coordinates': [-87.65289476...",NaN,NaN,NaN,NaN,NaN
35113,8146 S ST LAWRENCE AVE,8146,S,ST LAWRENCE,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.74614699608132,-87.61008757367442,"{'type': 'Point', 'coordinates': [-87.61008757...",NaN,NaN,NaN,NaN,NaN
35114,636 W SCHUBERT AVE,636,W,SCHUBERT,AVE,{'url': 'https://data.cityofchicago.org/resour...,{'url': 'https://data.cityofchicago.org/resour...,41.931720173523466,-87.64599787902512,"{'type': 'Point', 'coordinates': [-87.64599787...",NaN,NaN,NaN,NaN,NaN


In [252]:
df.complaints[0]

{'url': 'https://data.cityofchicago.org/resource/fypr-ksnz.csv?street_number=1%20and%20direction=%27S%27%20and%20street_name=%27FINANCIAL%27%20and%20street_type=%27PL%27',
 'description': 'Y'}

In [254]:
complaints['complaints'] = pd.Series()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
